# TQQQ Demo

In [13]:
import pandas as pd

## Simulating Leveraged Adjusted Close Prices

The code loads and processes simulated leveraged adjusted close prices for a given equity. Here's a breakdown:
- The function `simulate_data.process_leveraged_data()` downloads historical price data for a specified ticker (e.g., "QQQ") and uses a leverage scalar (e.g., 3x) to determine the multiplier for leveraged returns.
- The function calculates daily percentage returns (`Daily Return`) from the adjusted close prices.
- Leveraged returns are simulated by multiplying the daily returns by the specified leverage scalar.
- These leveraged returns are then compounded to simulate the cumulative leveraged price (`Simulated Leveraged Price`).


In [1]:
# Load in the desired ticker
import simulate_data
data = simulate_data.process_leveraged_data('QQQ', 3)
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,Leveraged Return,Simulated Leveraged Price
0,1999-03-10,51.12500,51.156250,50.28125,51.0625,43.417282,5232000,NaN,0.000000,43.417282
1,1999-03-11,51.43750,51.734375,50.31250,51.3125,43.629856,9688600,0.004896,0.014688,44.055004
2,1999-03-12,51.12500,51.156250,49.65625,50.0625,42.567024,8743600,-0.024360,-0.073081,40.835438
3,1999-03-15,50.43750,51.562500,49.90625,51.5000,43.789265,6369000,0.028713,0.086140,44.353001
4,1999-03-16,51.71875,52.156250,51.15625,51.9375,44.161274,4905800,0.008495,0.025486,45.483396


## Identifying Indices of All-Time Highs

This code identifies the indices where the adjusted close price and the simulated leveraged price reach new all-time highs to be used later in analysis.

In [8]:
# Grab Indices where the 'Adj Close' is the new all time high
ath_indices_baseline = []
current_ath = -9999
for idx, value in data['Adj Close'].items():
    if value > current_ath:
        current_ath = value
        ath_indices_baseline.append(idx)

# Grab Indices where the 'Simulated Leveraged Price' is the new all time high
ath_indices_leveraged  = []
current_ath = -9999
for idx, value in data['Simulated Leveraged Price'].items():
    if value > current_ath:
        current_ath = value
        ath_indices_leveraged .append(idx)

In [12]:
data.loc[ath_indices_leveraged].head()

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Return,Leveraged Return,Simulated Leveraged Price
0,1999-03-10,51.12500,51.156250,50.281250,51.0625,43.417282,5232000,NaN,0.000000,43.417282
1,1999-03-11,51.43750,51.734375,50.312500,51.3125,43.629856,9688600,0.004896,0.014688,44.055004
3,1999-03-15,50.43750,51.562500,49.906250,51.5000,43.789265,6369000,0.028713,0.086140,44.353001
4,1999-03-16,51.71875,52.156250,51.156250,51.9375,44.161274,4905800,0.008495,0.025486,45.483396
6,1999-03-18,51.50000,52.593750,51.484375,52.5625,44.692711,4848400,0.019394,0.058182,47.087225


In [34]:
len(ath_indices_baseline)

407

Calculate returns for investments made at all-time highs given a holding period of Z days.

In [29]:
def calculate_ath_returns(data, ath_indices, holding_period):
    returns = []
    dates = []
    for idx in ath_indices:
        if idx + holding_period < len(data):
            entry_price = data['Adj Close'].iloc[idx]
            exit_price = data['Adj Close'].iloc[idx + holding_period]
            return_pct = (exit_price - entry_price) / entry_price
            returns.append(return_pct)
            dates.append(data.loc[idx, 'Date'])
    return pd.DataFrame({'ATH_Index': ath_indices[:len(returns)], 'Date': dates, 'Return': returns})

# Example usage
returns_df = calculate_ath_returns(data, ath_indices_baseline, holding_period=1000)  # Z-day holding
returns_df.Return.describe()


count    305.000000
mean       0.757366
std        0.775317
min       -0.708506
25%        0.639362
50%        0.881788
75%        1.455870
max        1.679365
Name: Return, dtype: float64

In [30]:
returns_df

,ATH_Index,Date,Return
0,0,1999-03-10,-0.520196
1,1,1999-03-11,-0.521559
2,3,1999-03-15,-0.523495
3,4,1999-03-16,-0.537713
4,6,1999-03-18,-0.539025
...,...,...,...
300,5472,2020-12-04,0.709499
301,5473,2020-12-07,0.686490
302,5474,2020-12-08,0.695569
303,5480,2020-12-16,0.733734
